### 統計學習與深度學習
### Homework 5

請將IPYNB檔與IPYNB Export之HTML檔上傳至COOL作業區。回答作業時建議使用 "三明治" 答題法。也就是說，先說明要做什麼，然後列出程式碼與結果，最後說明這些結果的意義。作業自己做。嚴禁抄襲。不接受紙本繳交，不接受遲交。請以英文或中文作答。

這個作業將要練習影像分類的問題。影像分類是CNN模型的強項，我們的任務是區分照片中主角穿的上衣類型。這個問題在不同的情境下有不同的難度。在`Dive into Deep Learning`中有類似的問題，但是處理較"乾淨"的影像。這次作業的資料來自街拍影像，因此分類的困難度較高。

我們這次作業的任務，是依照照片中人物的上衣，區分以下類別:
* blazer
* cardigan
* coat
* jacket

下面列出這四個類別的範例訓練資料。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import random

random.seed(12)
dataset = ['train', 'valid', 'test']
labels = ['blazer', 'cardigan', 'coat', 'jacket']
DATA_PATH = "/content/drive/My Drive/photos"
for i in range(4):
    print("Label = ", labels[i])
    basepath = os.path.join(DATA_PATH+"/train", labels[i], "*.jpg") # DATA_PATH+'/msd_full.pickle'
    cand_fn = glob.glob(basepath)
    for afn in random.choices(cand_fn, k = 3):    
        img = Image.open(afn)
        plt.imshow(img)
        plt.show()

### 資料
資料在`photos`資料夾。已經區分好訓練(train)、校正(valid)、測試(test)資料。下一層則是依照圖片的標籤分資料夾存放，因此有四個資料夾，分別是blazer, cardigan, coat, jacket。一張圖片只會屬於一個類別。

### Q1
(5%) 列出train, valid, test的總照片數，以及各類別的照片數與比率。在還沒進行模型訓練與評估前，你認為各類別相對的準確率的大小關係為何?

### Q2
(35%) 使用Resnet50建構圖片分類模型。將最後一層的Fully Connected Layer輸出維度改成4以符合本題任務需求。除了最後一層以外，使用torchvision提供的pretrained weights (`torchvision.models.resnet50(pretrained=True)`)初始化模型權重。使用train資料訓練模型，以valid資料決定Early Stopping的Epoch。Early Stopping的Patient參數設為20 Epochs。Batch size設為32。每一個Epoch計算一次Valid Loss，並記錄Valid Loss最低的模型。模型訓練最多200個Epochs。使用最佳模型在test資料計算模型Accuracy, Confusion Matrix, 與Per-class Accuracy。你應該要考慮SGD與ADAM兩種最佳化演算法。調整超參數以達到最好的Valid Loss。

由於圖片的解析度較高，模型訓練前須將解析度調整(Resize)成較短邊為256像素的照片，然後隨機取大小為224x224的影像。接著隨機水平翻轉(Horizontal Flip)、隨機旋轉-20度到20度，並依照Pretrained ResNet的要求調整RGB的均數與標準差。
測試資料(Valid and Test)亦須先將解析度調整(Resize)成較短邊為256像素的照片，然後取圖片中心224x224的影像。

得到Per-Class Accuracy之後，請討論與Q1預期的差異與可能原因。

提示: 
* Pytorch Resnet pretrained model的說明請見 <https://pytorch.org/hub/pytorch_vision_resnet/>
* 本題的Test Accuracy應高於78%。

### Q3
(30%) 使用Resnet50建構圖片分類模型。將最後一層的Fully Connected Layer輸出維度改成4以符合本題任務需求。除了最後一層以外，使用torchvision提供的pretrained weights (`torchvision.models.resnet50(pretrained=True)`)初始化模型權重。模型訓練時固定除了最後一層以外的其他權重。也就是說，模型訓練只會調整最後一層Fully Connected Layer。圖片前處理與前題一致。

使用train訓練模型，以valid決定early stopping的epoch。Early stopping的patient參數為20 epochs。Batch size設為32。紀錄valid loss最低的模型，並在test中計算模型Accuracy, Confusion Matrix, 與Per-class Accuracy。你應該要考慮SGD與ADAM兩種最佳化演算法。調整超參數以達到最好的valid loss。

### Q4
(20%) 使用Resnet50建構圖片分類模型。將最後一層的fully connected layer輸出維度改成4以符合本題任務需求。圖片前處理與前題一致。不使用預訓練權重初始化模型。使用train訓練模型，以valid決定early stopping的epoch。Early stopping的patient參數為20 epochs。Batch size設為32。紀錄valid loss最低的模型，並在test中計算模型Accuracy, Confusion Matrix, 與Per-class Accuracy。你應該要考慮SGD與ADAM兩種最佳化演算法。調整超參數以達到最好的valid loss。

### Q5
(10%) 統整併討論Q2-Q4的預測能力。說明你的觀察。

In [3]:
# Q1
import numpy as np
import pandas as pd

num = np.array([])

for i in dataset:
  print("Dataset = ", i)
  DATA_PATH = "/content/drive/My Drive/photos/" + i
  for j in range(4):
    basepath = os.path.join(DATA_PATH, labels[j], "*.jpg") # DATA_PATH+'/msd_full.pickle'
    cand_fn = len(glob.glob(basepath))
    print(" Label = ", labels[j], " : %d photos" % cand_fn)
    num = np.append(num, cand_fn)

num_train = np.copy(num[0:4])
num_valid = np.copy(num[4:8])
num_test = np.copy(num[8:12])

Dataset =  train
 Label =  blazer  : 97 photos
 Label =  cardigan  : 237 photos
 Label =  coat  : 296 photos
 Label =  jacket  : 411 photos
Dataset =  valid
 Label =  blazer  : 7 photos
 Label =  cardigan  : 36 photos
 Label =  coat  : 27 photos
 Label =  jacket  : 35 photos
Dataset =  test
 Label =  blazer  : 9 photos
 Label =  cardigan  : 42 photos
 Label =  coat  : 43 photos
 Label =  jacket  : 52 photos


In [51]:
print("There are %d photos in train" % int(sum(num_train)))
print("There are %d photos in valid" % int(sum(num_valid)))
print("There are %d photos in test" % int(sum(num_test)))

df = pd.DataFrame([num_train, num_valid, num_test], dataset, labels)
df_ratio = pd.DataFrame([num_train/1041, num_valid/105, num_test/146], dataset, labels)

There are 1041 photos in train
There are 105 photos in valid
There are 146 photos in test


In [5]:
df

,blazer,cardigan,coat,jacket
train,97.0,237.0,296.0,411.0
valid,7.0,36.0,27.0,35.0
test,9.0,42.0,43.0,52.0


In [6]:
df_ratio

,blazer,cardigan,coat,jacket
train,0.093180,0.227666,0.284342,0.394813
valid,0.066667,0.342857,0.257143,0.333333
test,0.061644,0.287671,0.294521,0.356164


由於test資料集裡面cardigan,coat,jacket的數量比blazer多不少，因此預估預測blazer的準確程度不高，而其中又以jacket的預測可能最準確

# **A2**

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

data_transforms = {
    'train': transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),

    'valid': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),

    'test': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])}



In [8]:
DATA_PATH = "/content/drive/My Drive/photos"
# 建立 Dataset
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_PATH, x), data_transforms[x])
                  for x in ['train', 'valid', 'test']}

# 建立 DataLoader
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
               shuffle=True, num_workers=4)
               for x in ['train', 'valid', 'test']}

In [9]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
print(dataset_sizes)

{'train': 1041, 'valid': 105, 'test': 146}


In [10]:
class_names = image_datasets['test'].classes
print(class_names)

['blazer', 'cardigan', 'coat', 'jacket']


In [11]:
# 若 CUDA 環境可用，則使用 GPU 計算，否則使用 CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [46]:
# 訓練模型用函數
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):

    # 記錄最佳模型
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 10
    patient = 0

    # 訓練模型主迴圈
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 對於每個 epoch，分別進行訓練模型與驗證模型
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 將模型設定為訓練模式
            else:
                model.eval()   # 將模型設定為驗證模式

            running_loss = 0.0
            running_corrects = 0

            # 以 DataLoader 載入 batch 資料
            for inputs, labels in dataloaders[phase]:
                # 將資料放置於 GPU 或 CPU
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 重設參數梯度（gradient）
                optimizer.zero_grad()

                # 只在訓練模式計算參數梯度
                with torch.set_grad_enabled(phase == 'train'):
                    # 正向傳播（forward）
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()  # 反向傳播（backward）
                        optimizer.step() # 更新參數

                # 計算統計值
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                # 更新 scheduler
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
            
            # 記錄最佳模型
            if phase == 'valid' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                patient = 0
                # print(patient)
            elif phase == 'valid' and epoch_loss >= best_loss:
                patient += 1
                # print(patient)

            if patient == 20:
                break
        
        
        print()
        if patient == 20:
            break
    # 輸出最佳準確度
    print('Best valid Acc: {:4f}'.format(best_acc), 'Best valid Loss: {:4f}'.format(best_loss))

    # 載入最佳模型參數
    model.load_state_dict(best_model_wts)
    return model

In [13]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 4)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
#使用SGD
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# 每 50 個 epochs 將 learning rate 降為原本的 0.1 倍
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.2844 Acc: 0.3910
valid Loss: 1.3103 Acc: 0.4095

Epoch 1/199
----------
train Loss: 1.1150 Acc: 0.5360
valid Loss: 1.0469 Acc: 0.5238

Epoch 2/199
----------
train Loss: 0.9854 Acc: 0.5889
valid Loss: 0.8617 Acc: 0.7143

Epoch 3/199
----------
train Loss: 0.8118 Acc: 0.6590
valid Loss: 0.8287 Acc: 0.6857

Epoch 4/199
----------
train Loss: 0.6649 Acc: 0.7464
valid Loss: 0.7423 Acc: 0.7238

Epoch 5/199
----------
train Loss: 0.5316 Acc: 0.8060
valid Loss: 0.6654 Acc: 0.7524

Epoch 6/199
----------
train Loss: 0.4654 Acc: 0.8415
valid Loss: 0.5647 Acc: 0.7905

Epoch 7/199
----------
train Loss: 0.3799 Acc: 0.8607
valid Loss: 0.5642 Acc: 0.7619

Epoch 8/199
----------
train Loss: 0.2943 Acc: 0.9107
valid Loss: 0.5436 Acc: 0.7429

Epoch 9/199
----------
train Loss: 0.2559 Acc: 0.9107
valid Loss: 0.5264 Acc: 0.8095

Epoch 10/199
----------
train Loss: 0.1895 Acc: 0.9443
valid Loss: 0.5367 Acc: 0.7810

Epoch 11/199
----------
train Loss: 0.1687 Acc: 0.951

In [14]:
# 使用模型進行預測，並顯示結果

import sklearn as sk
from sklearn import metrics

def eval_model(model, y_true = np.array([]), y_pred = np.array([])):
  was_training = model.training # 記錄模型之前的模式
  model.eval() # 將模型設定為驗證模式
  corrects = 0



  with torch.no_grad():
      # 以 DataLoader 載入 batch 資料
      for i, (inputs, labels) in enumerate(dataloaders['test']):
          # 將資料放置於 GPU 或 CPU
          inputs = inputs.to(device)
          labels = labels.to(device)

          # 使用模型進行預測
          outputs = model(inputs)
          # print(outputs)
          _, preds = torch.max(outputs, 1)
          # print(_, preds)
          # print(preds == labels.data)
          corrects += torch.sum(preds == labels.data)
          for j in range(inputs.size()[0]):

            y_true = np.append(y_true, class_names[labels.data[j]])
            y_pred = np.append(y_pred, class_names[preds[j]])
            

  print(corrects)
  model.train(mode=was_training) # 恢復模型之前的模式
  return(y_true, y_pred)

y_true, y_pred = eval_model(model_ft)

confusion_matrix = metrics.confusion_matrix(y_true, y_pred, labels=["blazer", "cardigan", "coat", "jacket"])

# print out confusion matrix
print("Confusion Matrix:\n", confusion_matrix)

tensor(114, device='cuda:0')
Confusion Matrix:
 [[ 7  0  1  1]
 [ 1 32  3  6]
 [ 1  5 35  2]
 [ 0 10  2 40]]


In [15]:
corrects = 0
for i in range(0, 4):
  corrects += confusion_matrix[i, i]

print("Accuracy = ", corrects/sum(num_test))
print("Per-class Accuracy of Blazer = ", confusion_matrix[0, 0]/sum(confusion_matrix[0, ]))
print("Per-class Accuracy of Cardigan = ", confusion_matrix[1, 1]/sum(confusion_matrix[1, ]))
print("Per-class Accuracy of Coat = ", confusion_matrix[2, 2]/sum(confusion_matrix[2, ]))
print("Per-class Accuracy of Jacket = ", confusion_matrix[3, 3]/sum(confusion_matrix[3, ]))

Accuracy =  0.7808219178082192
Per-class Accuracy of Blazer =  0.7777777777777778
Per-class Accuracy of Cardigan =  0.7619047619047619
Per-class Accuracy of Coat =  0.813953488372093
Per-class Accuracy of Jacket =  0.7692307692307693


In [19]:
model_ft_2 = models.resnet50(pretrained=True)
num_ftrs_2 = model_ft_2.fc.in_features
model_ft_2.fc = nn.Linear(num_ftrs_2, 4)
model_ft_2 = model_ft_2.to(device)

criterion = nn.CrossEntropyLoss()
#使用Adam
optimizer_ft = optim.Adam(model_ft_2.parameters(), lr=0.001, weight_decay=0.01)
# 每 50 個 epochs 將 learning rate 降為原本的 0.1 倍
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft_2 = train_model(model_ft_2, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.4561 Acc: 0.3794
valid Loss: 3.4585 Acc: 0.3714

Epoch 1/199
----------
train Loss: 1.2687 Acc: 0.3948
valid Loss: 3.5715 Acc: 0.3619

Epoch 2/199
----------
train Loss: 1.2925 Acc: 0.3862
valid Loss: 1.3041 Acc: 0.3333

Epoch 3/199
----------
train Loss: 1.2694 Acc: 0.3862
valid Loss: 1.2696 Acc: 0.3524

Epoch 4/199
----------
train Loss: 1.2659 Acc: 0.4015
valid Loss: 1.2452 Acc: 0.4000

Epoch 5/199
----------
train Loss: 1.2595 Acc: 0.4159
valid Loss: 1.2528 Acc: 0.3619

Epoch 6/199
----------
train Loss: 1.2744 Acc: 0.3958
valid Loss: 1.2578 Acc: 0.4095

Epoch 7/199
----------
train Loss: 1.2522 Acc: 0.4102
valid Loss: 1.2544 Acc: 0.3619

Epoch 8/199
----------
train Loss: 1.2689 Acc: 0.4150
valid Loss: 1.2964 Acc: 0.3333

Epoch 9/199
----------
train Loss: 1.2564 Acc: 0.3939
valid Loss: 1.4370 Acc: 0.3905

Epoch 10/199
----------
train Loss: 1.2430 Acc: 0.3996
valid Loss: 1.2584 Acc: 0.3905

Epoch 11/199
----------
train Loss: 1.2295 Acc: 0.426

In [40]:
y_true_2, y_pred_2 = eval_model(model_ft_2)

confusion_matrix_2 = metrics.confusion_matrix(y_true_2, y_pred_2, labels=["blazer", "cardigan", "coat", "jacket"])

# print out confusion matrix
print("Confusion Matrix:\n", confusion_matrix_2)

corrects_2 = 0
for i in range(0, 4):
  corrects_2 += confusion_matrix_2[i, i]

print("Accuracy = ", corrects_2/sum(num_test))
print("Per-class Accuracy of Blazer = ", confusion_matrix_2[0, 0]/sum(confusion_matrix_2[0, ]))
print("Per-class Accuracy of Cardigan = ", confusion_matrix_2[1, 1]/sum(confusion_matrix_2[1, ]))
print("Per-class Accuracy of Coat = ", confusion_matrix_2[2, 2]/sum(confusion_matrix_2[2, ]))
print("Per-class Accuracy of Jacket = ", confusion_matrix_2[3, 3]/sum(confusion_matrix_2[3, ]))

tensor(52, device='cuda:0')
Confusion Matrix:
 [[ 0  2  1  6]
 [ 0 24  2 16]
 [ 0 20  1 22]
 [ 0 18  7 27]]
Accuracy =  0.3561643835616438
Per-class Accuracy of Blazer =  0.0
Per-class Accuracy of Cardigan =  0.5714285714285714
Per-class Accuracy of Coat =  0.023255813953488372
Per-class Accuracy of Jacket =  0.5192307692307693


SGD的表現比Adam的表現好，Test Accuracy可以達到0.7808。其他Per-class Accuracy分別為：Per-class Accuracy of Blazer =  0.7778
Per-class Accuracy of Cardigan =  0.7619
Per-class Accuracy of Coat =  0.81
Per-class Accuracy of Jacket =  0.76923。與前面第一猜想Blazer的預測能力很差不同，可能原因為Blazer圖片較特別，容易與其他衣服種類分辨出來，因此模型也比較好預測Blazer的衣服。

-------------------------------------------------------------------------------

# **A3**

In [36]:
model_ft_3 = models.resnet50(pretrained=True)
num_ftrs_3 = model_ft_3.fc.in_features
model_ft_3.fc = nn.Linear(num_ftrs_3, 4)
model_ft_3 = model_ft_3.to(device)

ct = 0
for child in model_ft_3.children():
  if ct < 9:
    for param in child.parameters():
        param.requires_grad = False
  ct += 1

criterion = nn.CrossEntropyLoss()
#使用SGD
optimizer_ft = optim.SGD(model_ft_3.parameters(), lr=0.001, momentum=0.9)
# 每 50 個 epochs 將 learning rate 降為原本的 0.1 倍
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft_3 = train_model(model_ft_3, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.3074 Acc: 0.3689
valid Loss: 1.2464 Acc: 0.3333

Epoch 1/199
----------
train Loss: 1.1934 Acc: 0.4899
valid Loss: 1.1963 Acc: 0.4095

Epoch 2/199
----------
train Loss: 1.1438 Acc: 0.5034
valid Loss: 1.1293 Acc: 0.4381

Epoch 3/199
----------
train Loss: 1.1128 Acc: 0.5312
valid Loss: 1.0898 Acc: 0.4476

Epoch 4/199
----------
train Loss: 1.0769 Acc: 0.5418
valid Loss: 1.1363 Acc: 0.4667

Epoch 5/199
----------
train Loss: 1.0294 Acc: 0.5696
valid Loss: 1.0541 Acc: 0.5143

Epoch 6/199
----------
train Loss: 1.0310 Acc: 0.5639
valid Loss: 1.1423 Acc: 0.4571

Epoch 7/199
----------
train Loss: 1.0455 Acc: 0.5466
valid Loss: 1.1766 Acc: 0.4476

Epoch 8/199
----------
train Loss: 1.0101 Acc: 0.5677
valid Loss: 0.9910 Acc: 0.5714

Epoch 9/199
----------
train Loss: 0.9807 Acc: 0.5869
valid Loss: 0.9910 Acc: 0.6190

Epoch 10/199
----------
train Loss: 0.9536 Acc: 0.6263
valid Loss: 1.0007 Acc: 0.5238

Epoch 11/199
----------
train Loss: 0.9597 Acc: 0.594

In [41]:
y_true_3, y_pred_3 = eval_model(model_ft_3)

confusion_matrix_3 = metrics.confusion_matrix(y_true_3, y_pred_3, labels=["blazer", "cardigan", "coat", "jacket"])

# print out confusion matrix
print("Confusion Matrix:\n", confusion_matrix_3)

corrects_3 = 0
for i in range(0, 4):
  corrects_3 += confusion_matrix_3[i, i]

print("Accuracy = ", corrects_3/sum(num_test))
print("Per-class Accuracy of Blazer = ", confusion_matrix_3[0, 0]/sum(confusion_matrix_3[0, ]))
print("Per-class Accuracy of Cardigan = ", confusion_matrix_3[1, 1]/sum(confusion_matrix_3[1, ]))
print("Per-class Accuracy of Coat = ", confusion_matrix_3[2, 2]/sum(confusion_matrix_3[2, ]))
print("Per-class Accuracy of Jacket = ", confusion_matrix_3[3, 3]/sum(confusion_matrix_3[3, ]))

tensor(89, device='cuda:0')
Confusion Matrix:
 [[ 2  1  2  4]
 [ 0 23  5 14]
 [ 0  8 21 14]
 [ 0  7  2 43]]
Accuracy =  0.6095890410958904
Per-class Accuracy of Blazer =  0.2222222222222222
Per-class Accuracy of Cardigan =  0.5476190476190477
Per-class Accuracy of Coat =  0.4883720930232558
Per-class Accuracy of Jacket =  0.8269230769230769


In [42]:
model_ft_4 = models.resnet50(pretrained=True)
num_ftrs_4 = model_ft_4.fc.in_features
model_ft_4.fc = nn.Linear(num_ftrs_4, 4)
model_ft_4 = model_ft_4.to(device)

ct = 0
for child in model_ft_4.children():
  if ct < 9:
    for param in child.parameters():
        param.requires_grad = False
  ct += 1

criterion = nn.CrossEntropyLoss()
#使用SGD
optimizer_ft = optim.Adam(model_ft_4.parameters(), lr=0.001, weight_decay=0.01)
# 每 50 個 epochs 將 learning rate 降為原本的 0.1 倍
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft_4 = train_model(model_ft_4, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.2929 Acc: 0.3996
valid Loss: 1.1841 Acc: 0.4667

Epoch 1/199
----------
train Loss: 1.1857 Acc: 0.4697
valid Loss: 1.1087 Acc: 0.5333

Epoch 2/199
----------
train Loss: 1.0672 Acc: 0.5620
valid Loss: 1.0305 Acc: 0.5714

Epoch 3/199
----------
train Loss: 1.0227 Acc: 0.5696
valid Loss: 1.0032 Acc: 0.5333

Epoch 4/199
----------
train Loss: 1.0212 Acc: 0.5591
valid Loss: 1.0530 Acc: 0.5238

Epoch 5/199
----------
train Loss: 0.9970 Acc: 0.5629
valid Loss: 0.9990 Acc: 0.5524

Epoch 6/199
----------
train Loss: 0.9587 Acc: 0.5927
valid Loss: 1.0082 Acc: 0.5524

Epoch 7/199
----------
train Loss: 0.9827 Acc: 0.5850
valid Loss: 1.1863 Acc: 0.4762

Epoch 8/199
----------
train Loss: 1.0254 Acc: 0.5447
valid Loss: 1.0040 Acc: 0.5714

Epoch 9/199
----------
train Loss: 0.9539 Acc: 0.6081
valid Loss: 1.0766 Acc: 0.5429

Epoch 10/199
----------
train Loss: 0.9128 Acc: 0.6158
valid Loss: 0.9677 Acc: 0.5619

Epoch 11/199
----------
train Loss: 0.8930 Acc: 0.623

In [43]:
y_true_4, y_pred_4 = eval_model(model_ft_4)

confusion_matrix_4 = metrics.confusion_matrix(y_true_4, y_pred_4, labels=["blazer", "cardigan", "coat", "jacket"])

# print out confusion matrix
print("Confusion Matrix:\n", confusion_matrix_4)

corrects_4 = 0
for i in range(0, 4):
  corrects_4 += confusion_matrix_4[i, i]

print("Accuracy = ", corrects_4/sum(num_test))
print("Per-class Accuracy of Blazer = ", confusion_matrix_4[0, 0]/sum(confusion_matrix_4[0, ]))
print("Per-class Accuracy of Cardigan = ", confusion_matrix_4[1, 1]/sum(confusion_matrix_4[1, ]))
print("Per-class Accuracy of Coat = ", confusion_matrix_4[2, 2]/sum(confusion_matrix_4[2, ]))
print("Per-class Accuracy of Jacket = ", confusion_matrix_4[3, 3]/sum(confusion_matrix_4[3, ]))

tensor(90, device='cuda:0')
Confusion Matrix:
 [[ 3  1  0  5]
 [ 0 25  3 14]
 [ 1  8 23 11]
 [ 2  8  3 39]]
Accuracy =  0.6164383561643836
Per-class Accuracy of Blazer =  0.3333333333333333
Per-class Accuracy of Cardigan =  0.5952380952380952
Per-class Accuracy of Coat =  0.5348837209302325
Per-class Accuracy of Jacket =  0.75


# **A4**

In [45]:
model_ft_5 = models.resnet50(pretrained=False)
num_ftrs_5 = model_ft_5.fc.in_features
model_ft_5.fc = nn.Linear(num_ftrs_5, 4)
model_ft_5 = model_ft_5.to(device)


criterion = nn.CrossEntropyLoss()
#使用SGD
optimizer_ft = optim.SGD(model_ft_5.parameters(), lr=0.001, momentum=0.9)
# 每 50 個 epochs 將 learning rate 降為原本的 0.1 倍
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft_5 = train_model(model_ft_5, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.3729 Acc: 0.3266
valid Loss: 1.3677 Acc: 0.3238

Epoch 1/199
----------
train Loss: 1.3883 Acc: 0.3573
valid Loss: 1.7565 Acc: 0.3333

Epoch 2/199
----------
train Loss: 1.3680 Acc: 0.3554
valid Loss: 1.4298 Acc: 0.3333

Epoch 3/199
----------
train Loss: 1.3329 Acc: 0.3487
valid Loss: 1.8427 Acc: 0.3238

Epoch 4/199
----------
train Loss: 1.4449 Acc: 0.3449
valid Loss: 1.5501 Acc: 0.3238

Epoch 5/199
----------
train Loss: 1.3362 Acc: 0.3622
valid Loss: 1.3412 Acc: 0.3238

Epoch 6/199
----------
train Loss: 1.3975 Acc: 0.3746
valid Loss: 1.2827 Acc: 0.4000

Epoch 7/199
----------
train Loss: 1.3813 Acc: 0.3516
valid Loss: 1.4905 Acc: 0.2952

Epoch 8/199
----------
train Loss: 1.4598 Acc: 0.3228
valid Loss: 1.9516 Acc: 0.3238

Epoch 9/199
----------
train Loss: 1.3118 Acc: 0.3775
valid Loss: 1.5363 Acc: 0.2476

Epoch 10/199
----------
train Loss: 1.2686 Acc: 0.3833
valid Loss: 1.4651 Acc: 0.2952

Epoch 11/199
----------
train Loss: 1.2585 Acc: 0.375

In [47]:
y_true_5, y_pred_5 = eval_model(model_ft_5)

confusion_matrix_5 = metrics.confusion_matrix(y_true_5, y_pred_5, labels=["blazer", "cardigan", "coat", "jacket"])

# print out confusion matrix
print("Confusion Matrix:\n", confusion_matrix_5)

corrects_5 = 0
for i in range(0, 4):
  corrects_5 += confusion_matrix_5[i, i]

print("Accuracy = ", corrects_5/sum(num_test))
print("Per-class Accuracy of Blazer = ", confusion_matrix_5[0, 0]/sum(confusion_matrix_5[0, ]))
print("Per-class Accuracy of Cardigan = ", confusion_matrix_5[1, 1]/sum(confusion_matrix_5[1, ]))
print("Per-class Accuracy of Coat = ", confusion_matrix_5[2, 2]/sum(confusion_matrix_5[2, ]))
print("Per-class Accuracy of Jacket = ", confusion_matrix_5[3, 3]/sum(confusion_matrix_5[3, ]))

tensor(44, device='cuda:0')
Confusion Matrix:
 [[ 0  6  3  0]
 [ 0 31 11  0]
 [ 0 30 13  0]
 [ 0 41 11  0]]
Accuracy =  0.3013698630136986
Per-class Accuracy of Blazer =  0.0
Per-class Accuracy of Cardigan =  0.7380952380952381
Per-class Accuracy of Coat =  0.3023255813953488
Per-class Accuracy of Jacket =  0.0


In [48]:
model_ft_6 = models.resnet50(pretrained=False)
num_ftrs_6 = model_ft_6.fc.in_features
model_ft_6.fc = nn.Linear(num_ftrs_6, 4)
model_ft_6 = model_ft_6.to(device)

criterion = nn.CrossEntropyLoss()
#使用SGD
optimizer_ft = optim.Adam(model_ft_6.parameters(), lr=0.001, weight_decay=0.01)
# 每 50 個 epochs 將 learning rate 降為原本的 0.1 倍
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft_6 = train_model(model_ft_6, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.5558 Acc: 0.3535
valid Loss: 1.3263 Acc: 0.4000

Epoch 1/199
----------
train Loss: 1.3441 Acc: 0.3602
valid Loss: 1.2779 Acc: 0.3810

Epoch 2/199
----------
train Loss: 1.2922 Acc: 0.3967
valid Loss: 1.2412 Acc: 0.3714

Epoch 3/199
----------
train Loss: 1.3189 Acc: 0.3785
valid Loss: 1.3791 Acc: 0.3333

Epoch 4/199
----------
train Loss: 1.2966 Acc: 0.3958
valid Loss: 1.5074 Acc: 0.2571

Epoch 5/199
----------
train Loss: 1.3006 Acc: 0.4073
valid Loss: 1.9402 Acc: 0.2952

Epoch 6/199
----------
train Loss: 1.3429 Acc: 0.3794
valid Loss: 2.1693 Acc: 0.3238

Epoch 7/199
----------
train Loss: 1.3006 Acc: 0.3862
valid Loss: 1.2825 Acc: 0.3333

Epoch 8/199
----------
train Loss: 1.2811 Acc: 0.3881
valid Loss: 1.3165 Acc: 0.3333

Epoch 9/199
----------
train Loss: 1.2812 Acc: 0.3948
valid Loss: 1.2640 Acc: 0.3333

Epoch 10/199
----------
train Loss: 1.2455 Acc: 0.4140
valid Loss: 1.2858 Acc: 0.3905

Epoch 11/199
----------
train Loss: 1.2393 Acc: 0.417

In [50]:
y_true_6, y_pred_6 = eval_model(model_ft_6)

confusion_matrix_6 = metrics.confusion_matrix(y_true_6, y_pred_6, labels=["blazer", "cardigan", "coat", "jacket"])

# print out confusion matrix
print("Confusion Matrix:\n", confusion_matrix_6)

corrects_6 = 0
for i in range(0, 4):
  corrects_6 += confusion_matrix_6[i, i]

print("Accuracy = ", corrects_6/sum(num_test))
print("Per-class Accuracy of Blazer = ", confusion_matrix_6[0, 0]/sum(confusion_matrix_6[0, ]))
print("Per-class Accuracy of Cardigan = ", confusion_matrix_6[1, 1]/sum(confusion_matrix_6[1, ]))
print("Per-class Accuracy of Coat = ", confusion_matrix_6[2, 2]/sum(confusion_matrix_6[2, ]))
print("Per-class Accuracy of Jacket = ", confusion_matrix_6[3, 3]/sum(confusion_matrix_6[3, ]))

tensor(56, device='cuda:0')
Confusion Matrix:
 [[ 0  4  0  5]
 [ 0 23  0 19]
 [ 0 19  0 24]
 [ 0 16  3 33]]
Accuracy =  0.3835616438356164
Per-class Accuracy of Blazer =  0.0
Per-class Accuracy of Cardigan =  0.5476190476190477
Per-class Accuracy of Coat =  0.0
Per-class Accuracy of Jacket =  0.6346153846153846


# **A5**

在第二題當中，使用SGD比使用Adam更好，而在各類服飾方面沒有像第一題預期一樣Blazer會最差，每個的per-class accuracy表現都差不多，有75%以上。

第三題freeze除了fc層以外的權重後，發現使用SGD的預測能力沒有第二題好，而且很明顯數量最多的Jacket的預測準確率 > Cardigan, Coat的 > Blazer的。但使用Adam的Accuracy相較於第二題提升了不少，甚至還比使用SGD大一點點，而且在per-class accuracy的落差沒有那麼大。

第四題捨棄掉pretrained_weight後，發現兩種optimizer的表現都變差很多，而且使用Adam的預測能力比使用SGD好上不少。使用這兩種optimizer預測也有一個共同的地方，就是只會將表現聚焦在預測兩種衣服，像SGD將所有衣服只有分成Cardigan或者Coat，而Adam則是將幾乎所有衣服分類為Cardigan或者Jacket。